In [0]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [0]:
# authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
import tarfile
# shared link to data archive file
# https://drive.google.com/open?id=1iqg3npFMm-7EeqDbZbwOHJbms01lAjbC 

  
# authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# download 
download = drive.CreateFile({'id': '1iqg3npFMm-7EeqDbZbwOHJbms01lAjbC'})
download.GetContentFile('aclImdb_v1.tar.gz')

# unzip
if not os.path.exists('data'):
  os.makedirs("data")

tar = tarfile.open('aclImdb_v1.tar.gz', "r:gz")
tar.extractall('data/')
tar.close()

In [0]:
import zipfile
# shared link to script files
#https://drive.google.com/open?id=1uU0LE_rzp_jZr4D-kqrYHPt-AIgoAiFs
  
download_scr = drive.CreateFile({'id': '1uU0LE_rzp_jZr4D-kqrYHPt-AIgoAiFs'})
download_scr.GetContentFile('scripts.zip')

zip_ref = zipfile.ZipFile('scripts.zip', "r")
zip_ref.extractall()
zip_ref.close()

In [0]:
import utils
# load data

train_path = os.path.join("data", "aclImdb", "train")
test_path = os.path.join("data", "aclImdb", "test")

train = utils.loaddata(train_path)
test = utils.loaddata(test_path)

In [0]:
from sklearn.model_selection import train_test_split
# split train set into train and validation 
train_data, val_data, train_target, val_target = train_test_split(train[0], train[1], shuffle=True, test_size=0.3)
test_data = test[0]
test_target = test[1]

In [0]:
import preprocess
# preprocess data
prp = preprocess.TextPreprocessor()

# only tokenization
X_train_tok = [prp.tokenize_prepr(text) for text in train_data]
print('X_train_tok')
X_val_tok = [prp.tokenize_prepr(text) for text in val_data]
print('X_val_tok')
X_test_tok = [prp.tokenize_prepr(text) for text in test_data]
print('X_test_tok')

X_train_tok
X_val_tok
X_test_tok


In [0]:
#import models
reload(models)
# create model
m = models.Model()

# get tfidt features
X_train_tfidf_tok, tfidf_vect = m.tfidf_train(X_train_tok)
X_val_tfidf_tok = m.tfidf_trans(tfidf_vect, X_val_tok)
X_test_tfidf_tok = m.tfidf_trans(tfidf_vect, X_test_tok)

In [0]:
clf_regr_tok = m.log_regression_train(X_train_tfidf_tok, train_target)
m.check_acc(clf_regr_tok, X_val_tfidf_tok, val_target)

Accuracy:  0.8846666666666667
Cross validation score:  [0.88141239 0.864      0.87533333 0.86733333 0.88192128]


(0.8846666666666667,
 array([0.88141239, 0.864     , 0.87533333, 0.86733333, 0.88192128]))

In [0]:
clf_linear_svc_tok = m.linear_svc_train(X_train_tfidf_tok, train_target)
m.check_acc(clf_linear_svc_tok, X_val_tfidf_tok, val_target)

Accuracy:  0.8821333333333333
Cross validation score:  [0.8787475  0.86333333 0.86466667 0.86066667 0.88058706]


(0.8821333333333333,
 array([0.8787475 , 0.86333333, 0.86466667, 0.86066667, 0.88058706]))

In [0]:
clf_adaboost_tok = m.adaboost_train(X_train_tfidf_tok, train_target)
m.check_acc(clf_adaboost_tok, X_val_tfidf_tok, val_target)

In [0]:
clf_rf_tok = m.random_forest_train(X_train_tfidf_tok, train_target)
m.check_acc(clf_rf_tok, X_val_tfidf_tok, val_target)

In [0]:
clf_svc_tok = m.svc_train(X_train_tfidf_tok, train_target)
m.svc_train(clf_svc_tok, X_val_tfidf_tok, val_target)

In [0]:
!ls

aclImdb_v1.tar.gz  code.zip  preprocess.py   sample_data
adc.json	   data      preprocess.pyc  scripts.7z
